In [3]:
# http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/
# http://learningtensorflow.com/index.html
# http://suriyadeepan.github.io/2016-12-31-practical-seq2seq/

import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

In [4]:
with tf.variable_scope('one_cell') as scope:
    # One cell RNN input_dim (3) -> output_dim (5)
    hidden_size = 5
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    print(cell.output_size, cell.state_size)

    x_data = np.array([[[1, 2, 3]]], dtype=np.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

5 5
array([[[ 0.92823738, -0.92360902,  0.89258885,  0.99514693, -0.036708  ]]], dtype=float32)


In [5]:
with tf.variable_scope('two_sequances') as scope:
    # One cell RNN input_dim (3) -> output_dim (5). sequence: 2
    hidden_size = 5
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    x_data = np.array([[[1, 2, 3],
                        [4, 5, 6]]], dtype=np.float32)
    outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 0.5095908 ,  0.73586476,  0.17925251,  0.35411239, -0.96497387],
        [-0.28526765,  0.97455138,  0.97645795, -0.60268986, -0.99961406]]], dtype=float32)


In [7]:
with tf.variable_scope('3_batches') as scope:
    # One cell RNN input_dim (3) -> output_dim (5). sequence: 2
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    x_data = np.array([[[1, 2, 3],
                        [4, 5, 6]],

                       [[7, 8, 9],
                        [10, 11, 12]],

                       [[13, 14, 15],
                        [16, 17, 18]], ], dtype=np.float32)
    outputs, _states = tf.nn.dynamic_rnn(
        cell, x_data, sequence_length=[1,2,1], dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ -1.51391983e-01,   8.77759010e-02,   7.22011039e-03,
           3.41427065e-02,   1.61610141e-01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]],

       [[  3.34553033e-01,  -3.43819201e-01,   1.82304037e-04,
           1.33128243e-03,   6.40465990e-02],
        [  7.24730372e-01,  -5.43702006e-01,   1.41877981e-05,
           2.95121019e-04,   3.15410048e-02]],

       [[  6.80008650e-01,  -4.49257314e-01,   6.61273702e-07,
           1.35716846e-05,   1.15950974e-02],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]]], dtype=float32)


In [8]:
with tf.variable_scope('initial_state') as scope:
    # One cell RNN input_dim (3) -> output_dim (5). sequence: 2
    batch_size = 3
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)

    x_data = np.array([[[1, 2, 3],
                        [4, 5, 6]],

                       [[7, 8, 9],
                        [10, 11, 12]],

                       [[13, 14, 15],
                        [16, 17, 18]], ], dtype=np.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data,
                                         initial_state=initial_state, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[  7.10822791e-02,   1.63086936e-01,   8.90568942e-02,
          -1.24750927e-01,   2.00498208e-01],
        [  4.70112897e-02,   3.78391862e-01,   2.16491129e-02,
          -1.13115773e-01,   4.86367106e-01]],

       [[  4.97021899e-03,   4.12906766e-01,   3.03682452e-03,
          -1.65634304e-02,   5.91667116e-01],
        [  7.24130834e-04,   4.93083537e-01,   5.39101311e-04,
          -5.02394000e-03,   6.94591641e-01]],

       [[  1.03420047e-04,   4.54885125e-01,   7.90608392e-05,
          -6.74612937e-04,   7.19891727e-01],
        [  1.22139072e-05,   5.20960569e-01,   1.35747678e-05,
          -1.44462858e-04,   7.46716678e-01]]], dtype=float32)


In [9]:
# Create input data
x_data = np.arange(24, dtype=np.float32).reshape(2, 4, 3)
pp.pprint(x_data)  # batch, sequence_length, input size

array([[[  0.,   1.,   2.],
        [  3.,   4.,   5.],
        [  6.,   7.,   8.],
        [  9.,  10.,  11.]],

       [[ 12.,  13.,  14.],
        [ 15.,  16.,  17.],
        [ 18.,  19.,  20.],
        [ 21.,  22.,  23.]]], dtype=float32)


In [10]:
with tf.variable_scope('MultiRNNCell') as scope:
    # Make rnn
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    cell = rnn.MultiRNNCell([cell] * 3, state_is_tuple=True)

    # rnn in/out
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    print("dynamic rnn: ", outputs)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())  # batch size, unrolling (time), hidden_size

dynamic rnn:  Tensor("MultiRNNCell/rnn/transpose:0", shape=(2, 4, 5), dtype=float32)
array([[[-0.00081725, -0.00386185,  0.00659794, -0.0023129 ,  0.00170874],
        [ 0.00209906, -0.0158419 ,  0.01976709, -0.00896045,  0.007729  ],
        [ 0.00951714, -0.03161216,  0.03275521, -0.01422561,  0.01544347],
        [ 0.01877232, -0.04602807,  0.04267565, -0.01718889,  0.02373555]],

       [[ 0.00509292, -0.00707487,  0.00062936, -0.0039078 ,  0.0031513 ],
        [ 0.01397658, -0.01832587,  0.00351024, -0.00880949,  0.00907965],
        [ 0.02361906, -0.02915494,  0.00721703, -0.01298102,  0.01674998],
        [ 0.03231562, -0.03762392,  0.0104991 , -0.01586265,  0.02497061]]], dtype=float32)


In [11]:
with tf.variable_scope('dynamic_rnn') as scope:
    cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32,
                                         sequence_length=[1, 3]) # lentgh 1 for batch 1, lentgh 2 for batch 2
    print("dynamic rnn: ", outputs)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())  # batch size, unrolling (time), hidden_size

dynamic rnn:  Tensor("dynamic_rnn/rnn/transpose:0", shape=(2, 4, 5), dtype=float32)
array([[[  1.44729763e-01,   8.68124291e-02,   2.04916492e-01,
           2.09573627e-01,  -5.28967381e-02],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]],

       [[  6.69837773e-01,   6.55202800e-03,   5.15428901e-01,
           6.26000166e-01,   1.09414861e-03],
        [  7.37198949e-01,   2.45613419e-03,   7.56947041e-01,
           6.68402612e-01,   3.07928160e-04],
        [  7.48316824e-01,   7.88202335e-04,   8.19158375e-01,
           6.95138216e-01,   7.53321801e-05],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]]], dtype=float32)


In [12]:
with tf.variable_scope('bi-directional') as scope:
    # bi-directional rnn
    cell_fw = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
    cell_bw = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)

    outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, x_data,
                                                      sequence_length=[2, 3],
                                                      dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    pp.pprint(sess.run(outputs))
    pp.pprint(sess.run(states))

(   array([[[ -9.65677872e-02,   8.78366530e-02,  -4.63173576e-02,
           1.70627236e-01,  -4.50006984e-02],
        [ -2.88367927e-01,  -2.20536790e-03,  -2.47096375e-01,
           2.39439070e-01,  -6.13145940e-02],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]],

       [[ -2.34570522e-02,  -2.30789676e-01,  -8.99251401e-02,
           4.12213095e-02,  -5.51276535e-05],
        [ -2.83833817e-02,  -3.16335082e-01,  -1.49197981e-01,
           2.10800320e-02,  -5.10345890e-05],
        [ -2.71350183e-02,  -3.37247968e-01,  -1.85490116e-01,
           1.06039764e-02,  -4.90470229e-05],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]]], dtype=float32),
    array([[[  1.84126094e-01,  -2.18982220e-01,   1.39720485e-01,
           3.44079107e-01,  -6.569111

In [10]:
# flattern based softmax
hidden_size=3
sequence_length=4
batch_size=2
num_classes=5

pp.pprint(x_data) # hidden_size=3, sequence_length=4, batch_size=2
x_data = x_data.reshape(-1, hidden_size)
pp.pprint(x_data)

softmax_w = np.arange(15, dtype=np.float32).reshape(hidden_size, num_classes)
outputs = np.matmul(x_data, softmax_w)
outputs = outputs.reshape(-1, sequence_length, num_classes) # batch, seq, class
pp.pprint(outputs)

array([[[  0.,   1.,   2.],
        [  3.,   4.,   5.],
        [  6.,   7.,   8.],
        [  9.,  10.,  11.]],

       [[ 12.,  13.,  14.],
        [ 15.,  16.,  17.],
        [ 18.,  19.,  20.],
        [ 21.,  22.,  23.]]], dtype=float32)
array([[  0.,   1.,   2.],
       [  3.,   4.,   5.],
       [  6.,   7.,   8.],
       [  9.,  10.,  11.],
       [ 12.,  13.,  14.],
       [ 15.,  16.,  17.],
       [ 18.,  19.,  20.],
       [ 21.,  22.,  23.]], dtype=float32)
array([[[  25.,   28.,   31.,   34.,   37.],
        [  70.,   82.,   94.,  106.,  118.],
        [ 115.,  136.,  157.,  178.,  199.],
        [ 160.,  190.,  220.,  250.,  280.]],

       [[ 205.,  244.,  283.,  322.,  361.],
        [ 250.,  298.,  346.,  394.,  442.],
        [ 295.,  352.,  409.,  466.,  523.],
        [ 340.,  406.,  472.,  538.,  604.]]], dtype=float32)


In [11]:
# [batch_size, sequence_length, emb_dim ]
prediction1 = tf.constant([[[0, 1], [0, 1], [0, 1]]], dtype=tf.float32)
prediction2 = tf.constant([[[1, 0], [1, 0], [1, 0]]], dtype=tf.float32)
prediction3 = tf.constant([[[0, 1], [1, 0], [0, 1]]], dtype=tf.float32)

# [batch_size, sequence_length]
y_data = tf.constant([[1, 1, 1]])

# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

sequence_loss1 = tf.contrib.seq2seq.sequence_loss(prediction1, y_data, weights)
sequence_loss2 = tf.contrib.seq2seq.sequence_loss(prediction2, y_data, weights)
sequence_loss3 = tf.contrib.seq2seq.sequence_loss(prediction3, y_data, weights)

sess.run(tf.global_variables_initializer())
print("Loss1: ", sequence_loss1.eval(),
      "Loss2: ", sequence_loss2.eval(),
      "Loss3: ", sequence_loss3.eval())

Loss1:  0.313262 Loss2:  1.31326 Loss3:  0.646595
